# Chat server

In [ ]:
//| export

import type { Database } from "tinychat/db.ts";
import { TID } from "@atproto/common";
import { fetchView, seedMessages, waitForSync } from "tinychat/db.ts";
import {
  DeleteMembershipRecord,
  DeleteServerRecord,
  NewMembershipRecord,
  NewServerRecord,
} from "tinychat/firehose.ts";
import {
  ServerSummaryView,
  ServerView,
  validateServerSummaryView,
  validateServerView,
} from "tinychat/api/types/chat/tinychat/server/defs.ts";
import { TinychatAgent } from "tinychat/agent.ts";

export class Servers {
  constructor(protected db: Database) {}

  async createServer(
    { name, tc }: { name: string; tc: TinychatAgent },
  ): Promise<ServerView> {
    const { uri } = await waitForSync<{ uri: string }>({
      db: this.db,
      op: () => {
        return tc.chat.tinychat.core.server.create(
          {
            repo: tc.agent.assertDid,
          },
          {
            name,
            channels: [
              { name: "general", id: TID.nextStr() },
              { name: "random", id: TID.nextStr() },
              { name: "meta", id: TID.nextStr() },
            ],
          },
        );
      },
      sql: ({ uri }) => `SELECT uri FROM servers WHERE uri = '${uri}'`,
    });

    return (await this.getServers({ uris: [uri] }))[0];
  }

  async joinServer(
    { server, tc }: { server: string; tc: TinychatAgent },
  ): Promise<{ uri: string }> {
    return await waitForSync<{ uri: string }>({
      db: this.db,
      op: () => {
        return tc.chat.tinychat.core.membership.create(
          { repo: tc.agent.assertDid },
          {
            server,
            createdAt: new Date().toISOString(),
          },
        );
      },
      sql: ({ uri }) =>
        `SELECT uri FROM server_memberships WHERE uri = '${uri}'`,
    });
  }

  syncServer(server: NewServerRecord) {
    const createChannel = this.db.prepare(
      `INSERT INTO channels (id, name, server) VALUES (:id, :name, :server) ON CONFLICT(id, server) DO NOTHING`,
    );

    this.db.transaction(() => {
      this.db
        .prepare(
          `
        INSERT INTO servers (uri, name, creator) VALUES (
          :uri, :name, :creator
        )`,
        )
        .run({
          uri: server.uri,
          name: server.commit.record.name,
          creator: server.did,
        });

      // this.db
      //   .prepare(
      //     `INSERT INTO server_memberships (user, server) VALUES (
      //       :creator, :server
      //     ) ON CONFLICT(user, server) DO NOTHING`,
      //   )
      //   .run({ creator: server.did, server: server.uri });

      for (const channel of server.commit.record.channels) {
        createChannel.run({
          id: channel.id,
          name: channel.name,
          server: server.uri,
        });
      }
    })();

    // seed messages if needed
    if (Deno.env.get("SEED_MESSAGES_AFTER_SERVER_CREATION")) {
      setTimeout(() => {
        seedMessages({ db: this.db, server: server.uri });
      }, 1000);
    }
  }

  public createMembership(m: NewMembershipRecord) {
    this.db.prepare(
      `INSERT INTO server_memberships (user, server, uri) VALUES (
        :user, :server, :uri
      )`,
    ).run({
      uri: m.uri,
      user: m.did,
      server: m.commit.record.server,
    });
  }

  deleteServer(server: DeleteServerRecord) {
    this.db.prepare(`DELETE FROM servers WHERE uri = '${server.uri}'`).run();
  }

  deleteMembership(membership: DeleteMembershipRecord) {
    this.db
      .prepare(
        `DELETE FROM server_memberships WHERE user = '${membership.did}' AND server = '${membership.uri}'`,
      )
      .run();
  }

  public getServers({
    uris,
    did,
  }: // viewer,
    {
      uris?: string[] | undefined;
      did?: string | undefined;
      viewer?: string | undefined;
    }): ServerView[] {
    let baseWhere = "";
    if (uris && uris.length > 0) {
      baseWhere = `uri IN (${uris.map((u) => `'${u}'`).join(", ")})`;
    } else if (did) {
      baseWhere = `creator__did = '${did}'`;
    }

    // viewer ? `viewer = '${viewer}'` : "",
    const where = [baseWhere]
      .filter((q) => q)
      .join(" AND ")
      .trim();

    return fetchView<ServerView>({
      db: this.db,
      // sql: `SELECT * FROM ${viewer ? 'server_view_with_viewer' : 'server_view'} ${where ? `WHERE ${where}` : ""}`,
      sql: `SELECT * FROM  server_view ${where ? `WHERE ${where}` : ""}`,
      validate: validateServerView,
    });
  }

  public findServers(
    { query }: { query?: string | undefined },
  ): ServerSummaryView[] {
    console.log("Finding servers with query: ", query);
    return fetchView<ServerSummaryView>({
      db: this.db,
      sql: "SELECT * FROM  server_view",
      validate: validateServerSummaryView,
    });
  }
}

## Tests

In [ ]:
import { assertEquals } from "asserts";
import { getDatabase } from "tinychat/db.ts";
import { createDefaultTestUser, createTestUser } from "tinychat/core/users.ts";

const s = {
  did: "did:plc:ubdeopbbkbgedccgbum7dhsh",
  time_us: 1738230288999577,
  commit: {
    rev: "3lgx75ibpx22b",
    operation: "create",
    collection: "chat.tinychat.core.server",
    rkey: "3lgx75ib5fc2b",
    cid: "bafyreigoeopnd7knsghm4xdxfenvf4kwfxoio6yxjp6io7nt6a4wly3fvy",
    record: {
      $type: "chat.tinychat.core.server",
      name: "test-3lgx75hnhht2l",
      channels: [{ id: "3lgx75hnocm2l", name: "general" }],
    },
  },
  uri:
    "at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgx75ib5fc2b",
};

Deno.test("syncServer", () => {
  const db = getDatabase({ reset: true });
  const servers = new Servers(db);
  createDefaultTestUser({ db });
  servers.syncServer(s);
  // spot check db
  assertEquals(
    db.prepare(`SELECT * FROM servers`).all().length,
    1,
    "got a server",
  );
});

Deno.test("getServers", () => {
  const db = getDatabase({ reset: true });
  const viewer = "did:plc:uowoeopbbkyuyewdccgbum7dhsh";
  createTestUser({
    db,
    user: {
      did: viewer,
      handle: "bob.com",
    },
  });
  const servers = new Servers(db);

  createDefaultTestUser({ db });
  servers.syncServer(s);

  assertEquals(
    servers.getServers({}).length,
    1,
    "got a server using empty query",
  );
  assertEquals(
    servers.getServers({ did: s.did }).length,
    1,
    "got a server using did",
  );
  assertEquals(
    servers.getServers({ uris: [s.uri] }).length,
    1,
    "got a server using uri",
  );

  // with viewer option
  assertEquals(
    servers.getServers({ viewer: s.did }).length,
    1,
    "got a server using empty query with viewer",
  );
  assertEquals(
    servers.getServers({ did: s.did, viewer: s.did }).length,
    1,
    "got a server using did and viewer",
  );
  assertEquals(
    servers.getServers({ uris: [s.uri], viewer: s.did }).length,
    1,
    "got a server using uri and viewer",
  );

  // viewer who is NOT chat server admin
  // assertEquals(
  //   servers.getServers({ viewer }).length,
  //   1,
  //   "got a server using empty query with viewer"
  // );
});

Deno.test("findServers", () => {
  const db = getDatabase({ reset: true });
  const servers = new Servers(db);
  createDefaultTestUser({ db });
  servers.syncServer(s);
  assertEquals(
    servers.findServers({}).length,
    1,
    "found 1 server",
  );
});

In [ ]:
import { assert, assertEquals } from "asserts";
import { TestDatabase } from "tinychat/db.ts";

Deno.test("delete server", () => {
  const testServers = TestDatabase.setup();
  const db = testServers.db;
  const servers = new Servers(db);

  // spot check db before delete

  assertEquals(
    db.prepare(`SELECT * FROM servers`).all().length,
    2,
    "starting with 2 servers",
  );
  assertEquals(
    db.prepare(`SELECT * FROM server_memberships`).all().length,
    4,
    "starting with 4 memberships",
  );
  assertEquals(
    db.prepare(`SELECT * FROM channels`).all().length,
    4,
    "starting with 4 channels",
  );
  // make sure we have messages for the server
  assert(
    db.prepare(`SELECT * FROM messages WHERE server = '${TestDatabase.server}'`)
      .all().length > 10,
    "got a bunch of messages",
  );

  const rkey = TestDatabase.server.split("/").pop()!;

  servers.deleteServer({
    did: TestDatabase.user1,
    time_us: 1738269878791634,
    commit: {
      rev: "3lgydzdyarc2b",
      operation: "delete",
      collection: "chat.tinychat.core.server",
      rkey,
    },
    uri: TestDatabase.server,
  });

  // spot check db after delete
  assertEquals(
    db.prepare(`SELECT * FROM servers`).all().length,
    1,
    "1 server left",
  );
  // make sure server is gone
  assertEquals(
    db.prepare(`SELECT * FROM servers WHERE uri = '${TestDatabase.server}'`)
      .all().length,
    0,
    "server is gone",
  );
  // make sure there are no channels left
  assertEquals(
    db.prepare(`SELECT * FROM channels WHERE server = '${TestDatabase.server}'`)
      .all().length,
    0,
    "channels are gone",
  );
  // make sure there are no memberships left
  assertEquals(
    db.prepare(
      `SELECT * FROM server_memberships WHERE server = '${TestDatabase.server}'`,
    ).all().length,
    0,
    "memberships are gone",
  );
  // make sure there are no messages left
  assertEquals(
    db.prepare(`SELECT * FROM messages WHERE server = '${TestDatabase.server}'`)
      .all().length,
    0,
    "messages are gone",
  );
});

In [ ]:
// Deno.test("get servers includes channel message ts info", () => {
//   const messaging = TestMessaging.setup();
//   messaging.user1MessagesChannel1("hello world");

//   // base case with no viewer set

//   let servers = messaging.getServers({
//     uris: [TestMessaging.server],
//   });

//   assertEquals(servers.length, 1, "got 1 server");
//   assertEquals(servers[0].channels.length, 2, "got 2 channels");
//   assert(servers[0].channels[0].id === TestMessaging.channel1);
//   assert(
//     servers[0].channels[0].latestMessageReceivedTime,
//     "channel 1 has last message received time set"
//   );
//   assert(
//     typeof servers[0].channels[0].lastMessageReadTime === "undefined",
//     "no last message read time data available when viewer para is not set"
//   );
//   assert(
//     typeof servers[0].channels[1].latestMessageReceivedTime === "undefined",
//     "channel 2 does not have last message received time set"
//   );

//   // got a viewer

//   servers = messaging.getServers({
//     uris: [TestMessaging.server],
//     viewer: TestMessaging.user2,
//   });

//   assertEquals(servers.length, 1);
//   assertEquals(servers[0].channels.length, 2, "got 2 channels");
//   assert(servers[0].channels[0].id === TestMessaging.channel1);
//   assert(
//     servers[0].channels[0].latestMessageReceivedTime,
//     "channel 1 has last message received time set"
//   );
//   assert(
//     typeof servers[0].channels[0].lastMessageReadTime === "undefined",
//     "last message read time data is not set initially"
//   );

//   // mark all messages as read and recheck

//   messaging.markAllMessagesAsRead({
//     server: TestMessaging.server,
//     channel: TestMessaging.channel1,
//     user: TestMessaging.user2,
//   });

//   servers = messaging.getServers({
//     uris: [TestMessaging.server],
//     viewer: TestMessaging.user2,
//   });

//   assertEquals(servers.length, 1);
//   assertEquals(servers[0].channels.length, 2, "got 2 channels");
//   assert(servers[0].channels[0].id === TestMessaging.channel1);
//   assert(
//     servers[0].channels[0].lastMessageReadTime,
//     "last message read time data is set after marking all messages as read"
//   );
// });